## 根据产品研发投入,广告投入,市场营销投入来预测一家公司的利润.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


dataset = pd.read_csv('50_Startups.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

## 虚拟变量

对公司所在地进行处理,因为原来string类型的数据没有有向关系(无大小,程度描述),根据n个数据类,将数据编码为n维.
此时会出现一个变量陷阱,应该去除一列,这就去除了应用多元线性回归时"多重共线性"陷阱,即自变量之间有线性强关系.
多一个参数对模型的影响是,拟合效果过好,模型的表达能力过于强会表现为过拟合现象.

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])
onehotencoder = OneHotEncoder(categorical_features = [3])
X = onehotencoder.fit_transform(X).toarray()

X = X[:, 1:]

/home/hanxinle/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/hanxinle/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
y_pred = regressor.predict(X_test)

## 建立多元线性模型的过程

此过程建立模型后,需要对各个变量进行评估,去除对模型影响微小(P-v大于阈值)的自变量.常用的生成多元线性模型的方法有:反向淘汰,正向淘汰,双向淘汰,全保留,这里用的是反向淘汰.
summary()中看P那一列

In [10]:
import statsmodels.regression.linear_model as sm

X_train = np.append(arr = np.ones((40, 1)).astype(int), values = X_train, axis = 1)
X_opt = X_train [:, [0, 1, 2, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X_train [:, [0, 1, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X_train [:, [0, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X_train [:, [0, 3, 5]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()
X_opt = X_train [:, [0, 3]]
regressor_OLS = sm.OLS(endog = y_train, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9072
Date:                Thu, 10 Oct 2019   Prob (F-statistic):              0.347
Time:                        19:53:12   Log-Likelihood:                -480.61
No. Observations:                  40   AIC:                             965.2
Df Residuals:                      38   BIC:                             968.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.049e+05   8046.652     13.038      0.000    8.86e+04    1.21e+05
x1          1.295e+04   1.36e+04      0.952      0.347   -1.46e+04    4.05e+04
==============================================================================
Omnibus:                        0.065   Durbin-Watson:                   1.954
Prob(Omnibus):                  0.968   Jarque-Bera (JB):                0.117
Skew:                          -0.078   Prob(JB):                        0.943
Kurtosis:                       2.786   Cond. No.                         2.42
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""